In [1]:
import pandas as pd
import geopandas as gpd

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.options.display.max_colwidth = 99

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
### Algoritmo

# - Primero haber excluido CABA en caso 1991, 2001.

# Cargar base de datos y mergear por codigo
# (Deptos en Base del Censo son los mismo que tiene Rodriguez?)

# Chequear la superposicion de los mergeados -> pasan a OK

#lo que no esta OK y lo que no coincide codigo va todo a compararse en spatial join

# Se concatenan resultados

In [4]:
year = '2001'

### Codigos de Fracciones segun base de datos

In [5]:
path_ = '/media/miglesia/Elements1/CENSO_dirs/Censo_'+year

if year == '1991':
    censo = pd.read_csv(path_ + '/FRACCION.csv', ';'
                       ).merge(pd.read_csv(path_ + '/DEPTO.csv', ';')
                             ).merge(pd.read_csv(path_ + '/PROVIN.csv', ';'))
    censo = censo.rename(columns = {'PROVIN': 'IDPROV', 'DEPTO': 'IDDPTO', 'FRACCION': 'IDFRAC', 'RADIO': 'IDRADIO'})
    
else:
    censo = pd.read_csv(path_ + '/FRAC.csv', ';'
                  ).merge(pd.read_csv(path_ + '/DPTO.csv', ';')
                         ).merge(pd.read_csv(path_ + '/PROV.csv', ';'))
    

# df['COD_1991'] = df['IDPROV'].astype(str).str.zfill(2) + df['IDDPTO'].astype(str).str.zfill(3) \
# + df['IDFRAC'].astype(str).str.zfill(2) + df['IDRADIO'].astype(str).str.zfill(2)


## Este df tiene Antartida e Islas, que en el archivo de Rodriguez no estan.
# 94021 Antártida Argentina, 94028 Islas del Atlántico Sur
# Se mergean por codigo, igual...

# # Excluyo CABA, en censos1991, 2001
# censo = censo.loc[censo.PROV != 2]

## Adapto columnas para mergear
censo['PROV_'] = censo.IDPROV.astype(str).str.zfill(2)
censo['DEPTO_'] = censo.IDDPTO.astype(str).str.zfill(3)
censo['FRACC_'] = censo.IDFRAC.astype(str).str.zfill(2)

In [6]:
censo.head(2)

,FRAC_REF_ID,DPTO_REF_ID,IDFRAC,PROV_REF_ID,IDDPTO,DPTO,NOMDEPTO,CPV2001_REF_ID,IDPROV,PROV,NOMPROV,PROV_,DEPTO_,FRACC_
0,1,1,1,1,1,2001,D.E. I,1,2,2,CIUDAD AUTONOMA DE BUENOS AIRES,02,001,01
1,2,1,2,1,1,2001,D.E. I,1,2,2,CIUDAD AUTONOMA DE BUENOS AIRES,02,001,02


### Shapefile de Censo revisado por CONICET

In [7]:
CONICET_radios = gpd.read_file('./'+year+'_RADIOS ARGENTINA/')
CONICET_radios['geometry'] = CONICET_radios['geometry'].buffer(1) # Un metro de buffer para garantizar que no queden marcas entre radios
# CONICET_radios = CONICET_radios.to_crs({'init': "EPSG:4326"})

## El archivo de rodriguez de 2010 tiene un prolemita por el cual los codigos de deptos de la provincia de Formosa fueron sobreescritos con los codigos de fracciones. 
# Para constatar que afecta solamente a todas las entradas de formosa (PROV 34)
# (CONICET_radios['COD_2010'].str[2:5] == CONICET_radios.DEPTO_).groupby(CONICET_radios.PROV_).sum()/CONICET_radios.groupby(CONICET_radios.PROV_).size()
if year == '2010':    # Lo corregimos
    CONICET_radios['DEPTO_'] = CONICET_radios['COD_2010'].str[2:5]
    
    
# # # Excluyo CABA, en censos 1991, 2001
# if year in ['1991', '2001']:
#     CONICET_radios = CONICET_radios.loc[CONICET_radios.PROV_ != '02']

### Disolver a nivel fracciones o deptos

In [8]:
#  disuelto a nivel Fracciones
CONICET_fracs = CONICET_radios.dissolve(['PROV_', 'DEPTO_', 'FRACC_']).reset_index()
CONICET_dptos = CONICET_fracs.dropna(subset = ['geometry']).dissolve(['PROV_', 'DEPTO_']).reset_index()

## Quitamos las columnas de nivel radio, que deberian desaparecer al disolver:
columnas_radios = ['COD_'+year,'RADIO_','OBS_','CODAGLO_1','CODAGLO_2','CODAGLO_3','CODAGLO_4','CODAGLO_5']
CONICET_fracs = CONICET_fracs.drop(columnas_radios, 1)
CONICET_dptos = CONICET_dptos.drop(['FRACC_'] + columnas_radios, 1)

CONICET_dptos.tail()

,PROV_,DEPTO_,geometry
526,90,105,"POLYGON ((3578949.470242848 7037156.000402951, 3578901.479171186 7037157.000216943, 3578901.451..."
527,90,112,"POLYGON ((3577121.25 7062005.444252735, 3577121.25 7061933.5, 3577121.248460353 7061933.4445299..."
528,90,119,"POLYGON ((3572646.936829273 7032956.652368576, 3572646.930896829 7032956.634717789, 3572646.918..."
529,94,007,"POLYGON ((3424455.796535045 3954208.017978789, 3424456.998930047 3954182.046246762, 3424456.999..."
530,94,014,"(POLYGON ((3607155.497432193 3937078.428382806, 3607155.486186475 3937078.334361122, 3607155.46..."


### Asignar geometrias a los codigos de areas en el censo.

In [9]:
dptos_censo = censo[['PROV_','DEPTO_']].drop_duplicates().merge(CONICET_dptos[['PROV_','DEPTO_', 'geometry']], how = 'left')
dptos_censo = dptos_censo.dropna() # Antartida
dptos_censo.crs = CONICET_radios.crs

### Referencia IGN

In [10]:
ref_ = gpd.read_file('./../chequeo_geo/departamento_IGN/')
ref_ = ref_.loc[~ref_.in1.isin(['94028', '94021'])] # Excluir Antartida e Islas
ref_ = ref_.to_crs(CONICET_radios.crs)
ref_['PROV_'] = ref_['in1'].str[:2]
ref_['DEPTO_'] = ref_['in1'].str[2:]
ref_.head()

,gid,objeto,fna,gna,nam,in1,fdc,sag,geometry,PROV_,DEPTO_
0,31,Departamento,Partido de General Lavalle,Partido,General Lavalle,06336,ARBA - Gerencia de Servicios Catastrales,IGN,"POLYGON ((4329062.509609145 5941098.999804945, 4329064.399665998 5941096.142980715, 4329065.472...",06,336
1,32,Departamento,Partido de La Costa,Partido,La Costa,06420,ARBA - Gerencia de Servicios Catastrales,IGN,"POLYGON ((4320378.90661143 5861758.19121304, 4320302.467809119 5861797.370885292, 4320295.36175...",06,420
2,33,Departamento,Partido de Magdalena,Partido,Magdalena,06505,ARBA - Gerencia de Servicios Catastrales,IGN,"POLYGON ((4258724.680924041 6103473.496525175, 4258751.558785764 6103469.118520383, 4258770.759...",06,505
3,34,Departamento,Partido de Vicente López,Partido,Vicente López,06861,ARBA - Gerencia de Servicios Catastrales,IGN,"POLYGON ((4191463.218437184 6157673.635250164, 4191456.79243413 6157671.71945148, 4191450.96017...",06,861
4,91,Departamento,Departamento San Justo,Departamento,San Justo,14140,IDE Cordoba,IGN,"POLYGON ((3869405.818689292 6631659.425839691, 3869758.837882656 6630199.599318543, 3870316.053...",14,140


In [11]:
ref_provs = gpd.read_file('./../chequeo_geo/provincia_IGN/')
ref_provs = ref_provs.to_crs(CONICET_radios.crs)
ref_provs['PROV_'] = ref_provs['in1'].str[:2]
ref_provs['DEPTO_'] = ref_provs['in1'].str[2:]

In [12]:
### Quizas es mejor que la union venga luego de cierta informacion.

## Unir IGN y Censo

In [13]:
### Tengo cierta informacion.
## Los codigos que son iguales, implican que no hubo cambio de limites. Excepto en CABA, donde los codigos de INDEC de comunas van del 1 al 15, como los codigos de los antiguos DE.

# Entonces, puedo excluir todos los codigos 

In [14]:
crossw_ref = dptos_censo[['PROV_', 'DEPTO_']].merge(ref_[['PROV_', 'DEPTO_', 'in1', 'nam']], how = 'outer', indicator = True)

In [16]:
## 2001: Interesa CABA o deptos que fueron subdivididos
if year == '2001':
    crossw_ref = crossw_ref.loc[(crossw_ref.PROV_ == '02') | (crossw_ref._merge != 'both')]

In [17]:
## De ahi saco superposiciones, y hago interseccion de radios.

In [18]:
## Deptos viejos
crossw_censo_geo = crossw_ref[['PROV_','DEPTO_']].drop_duplicates().merge(dptos_censo)
crossw_censo_geo = gpd.GeoDataFrame(crossw_censo_geo)

## Del lado de los de IGN, que sean solo los aceptables.
crossw_IGN_geo = ref_.loc[(ref_.PROV_ == '02') | (ref_.in1.isin(crossw_ref.in1.values))]


In [19]:
crossw_geo = gpd.overlay(crossw_censo_geo[['PROV_','DEPTO_', 'geometry']], crossw_IGN_geo[['in1', 'geometry']], how='intersection')


In [20]:
## Ahora, hacer la interseccion espacial entre cada deptos del censo con los de IGN.

## De un lado los radios a ubicar, aquellos en los deptos viejos.
radios_ubicar = gpd.GeoDataFrame(crossw_ref[['PROV_','DEPTO_']].drop_duplicates().merge(CONICET_radios[['PROV_','DEPTO_', 'COD_2001', 'geometry']]))


In [21]:
radios_ubicar.head()

,PROV_,DEPTO_,COD_2001,geometry
0,02,001,020013201,"(POLYGON ((4199821.283069543 6148736.97618706, 4199821.377926784 6148736.992521097, 4199821.473..."
1,02,001,020010710,"POLYGON ((4199386.00421795 6147696.99885848, 4199386.054304958 6147696.998524398, 4199386.14909..."
2,02,001,020010706,"POLYGON ((4197838.251689927 6148255.163349104, 4197838.292893219 6148255.207106781, 4197852.792..."
3,02,001,020010701,"POLYGON ((4197465.257097513 6148008.346698114, 4197465.189907974 6148008.276102916, 4197465.116..."
4,02,001,020010702,"POLYGON ((4197463.737490864 6148009.646977448, 4197463.742902487 6148009.653301886, 4197517.242..."


In [24]:
## Intersectar los radios de CABA y deptos subdivididos con los polis de IGS.
inters = gpd.overlay(radios_ubicar, crossw_IGN_geo[['in1', 'geometry']], how='intersection')
inters['area'] = inters.area

In [25]:
# Quedarse con el que mas interseca.
## La fraccion se asigna al depto donde este mayor parte de su area.
radios_in_depto = inters.groupby(['PROV_', 'DEPTO_', 'COD_2001']).apply(lambda x: x.nlargest(1, 'area')).reset_index(drop = True)[['PROV_', 'DEPTO_', 'COD_2001', 'in1']]

KeyboardInterrupt: 

In [ ]:
radios_in_depto.head()

In [ ]:
# for i, row in crossw_geo[['PROV_','DEPTO_']].drop_duplicates().iterrows():
    
#     rads = radios_in_depto.loc[(radios_in_depto.PROV_ == row.PROV_) & (radios_in_depto.DEPTO_ == row.DEPTO_)]


for i, row in crossw_geo[['PROV_']].drop_duplicates().iterrows():
    rads = radios_in_depto.loc[(radios_in_depto.PROV_ == row.PROV_)]
    rads_gdf = gpd.GeoDataFrame(rads.merge(CONICET_radios[['COD_2001', 'geometry']]))
    
    fig, ax = plt.subplots(1, figsize = (10, 10))
    
    rads_gdf.plot('in1', ax = ax, alpha = .5, edgecolor = 'w', lw = .4)
    
    ref_.loc[ref_.PROV_ == row.PROV_].plot(color = 'None', edgecolor = 'g', ax = ax, lw = .5)

    limits = gpd.GeoDataFrame(rads.merge(CONICET_radios[['COD_2001', 'geometry']])).dissolve('in1')
    limits.plot(color = 'None', edgecolor = 'r', ax = ax, lw = .5)
    
    # plot bounds
    x0, y0, x1, y1 = rads_gdf.total_bounds # minx, miny, maxx, maxy
    ax.set_xlim(x0, x1); ax.set_ylim(y0, y1)
    ax.axis('off')

    plt.show()

In [ ]:
xx

In [ ]:
spat_join = fracs_in_depto.groupby(['PROV_', 'DEPTO_', 'in1']).size().reset_index()
spat_join = spat_join.rename(columns = {0: 'n_fracs_in_depto'})

### Si en result los codigos son iguales, asociarlas.
## Esto es porque los DPTOS que se subdividen cambian el codigo. 
## Ademas, asi se evitan asociaciones espurias debidas a desalineamientos. La excepcion son las comunas de CABA, pero con esos no hay problemas de desalineaminto

dpto_codes_match_confirmados_2 = spat_join.loc[(spat_join.PROV_ + spat_join.DEPTO_) == spat_join.in1].drop('n_fracs_in_depto', 1) 
## Con que una fraccion este en un in1 del mismo codigo, ya todas las fracciones vienen asignadas a ese codigo en comun.

dpto_codes_match_confirmados_3 = pd.concat([dpto_codes_match_confirmados_1, dpto_codes_match_confirmados_2])

## Usar las intersecciones en los casos donde no coinciden los codigos
fracs_in_depto_ = fracs_in_depto.loc[~fracs_in_depto['in1'].isin(dpto_codes_match_confirmados_3['in1'])]
# ### Las Fracciones que quedan son de Chascomus, TdF y CABA

In [ ]:
spat_join

In [ ]:
## Primero se hace un merge por codigos
## Despues se mira la interseccion. Si hay algo cercano a un 1 a 1 ya esta.

In [ ]:
### ETAPA 1: merge por codigos.
## Se une por codigos y se mira si hay interseccion mutua

dpto_codes = dptos_censo[['PROV_', 'DEPTO_', 'geometry']].merge(ref_[['PROV_', 'DEPTO_', 'geometry']], on = ['PROV_', 'DEPTO_'], how = 'outer', indicator = True)

### Los codigos que hayan coincidido se chequean por interseccion espacial. 

dpto_codes_match = gpd.GeoDataFrame(dpto_codes.loc[dpto_codes._merge == 'both'])
inters_ = gpd.GeoSeries(dpto_codes_match['geometry_x']).intersection(gpd.GeoSeries(dpto_codes_match['geometry_y']))

dpto_codes_match['test1'] = (gpd.GeoSeries(dpto_codes_match['geometry_x']).area/inters_.area)
dpto_codes_match['test2'] = (gpd.GeoSeries(dpto_codes_match['geometry_y']).area/inters_.area)

In [ ]:
# gpd.GeoDataFrame(merged_geom.loc[merged_geom._merge != 'both'])

dpto_codes._merge.value_counts()

In [ ]:
### Tenemos los DEPTOS que se unieron en 1ra etapa. (la mayoria)
dpto_codes_match_confirmados_1 = dpto_codes_match.loc[(abs(dpto_codes_match.test1 - 1) < .1) & (abs(dpto_codes_match.test2 - 1) < .1)][['PROV_', 'DEPTO_']]
dpto_codes_match_confirmados_1['in1'] = dpto_codes_match_confirmados_1.PROV_ + dpto_codes_match_confirmados_1.DEPTO_

# Los radios de estos deptos se pueden unir de CONICET a IGN a partir de la clave.
dpto_codes_match_confirmados_1.shape

# En estos casos, los codigos de DPTO que usa el indec en Censo son los mismos que usa IGN

In [ ]:
## Para ver los confirmados, chequear nombres, etc

# gdf = dpto_codes_match_confirmados_1.merge(ref_[['PROV_', 'DEPTO_', 'nam', 'geometry']], how = 'outer', indicator = True)

# fig, ax = plt.subplots(1, figsize = (5, 10))

# gpd.GeoDataFrame(gdf.loc[gdf._merge == 'both']).plot(ax = ax, color = 'g', edgecolor = 'k', lw = 0)
# gpd.GeoDataFrame(gdf.loc[gdf._merge != 'both']).plot(ax = ax, color = 'r', edgecolor = 'k', lw = 0)

# plt.show()

In [ ]:
## En cambio, en estos casos hay alguna diferencia entre las geometrias, de modo que se van a unir espacialmente.
ok_vals = dpto_codes_match_confirmados_1.in1.values

In [ ]:
xx

In [ ]:
## Obtener las fracciones que NO estan en ok_vals
fracs_ubicar = CONICET_fracs.loc[~(CONICET_fracs.PROV_ + CONICET_fracs.DEPTO_).isin(ok_vals)].sort_values(['PROV_','DEPTO_','FRACC_'])

## Son todos los de CABA porque IGN usa otro codigo para las comunas, ademas de algunos del interior donde hubo cambios, (eg. Chascomus Lezama, TdF)
fracs_ubicar.shape

In [ ]:
### Puede haber algunas fracciones sin poligono. Esto va a pasar para Antartida e islas, y tambien para alguna fraccion perdida. 
# Las dejo perder?

display(fracs_ubicar.loc[fracs_ubicar.geometry.isna()])
# fracs_ubicar.loc[(fracs_ubicar.PROV_ == '86') & (fracs_ubicar.DEPTO_ == '063')]

fracs_ubicar = fracs_ubicar.loc[fracs_ubicar.geometry.notna()]

In [ ]:
## Como no son muchas, se pueden hacer sin loop provincial.

inters = gpd.overlay(fracs_ubicar, ref_[['in1', 'geometry']], how='intersection') 
inters['area'] = inters.area

## La fraccion se asigna al depto donde este mayor parte de su area.
fracs_in_depto = inters.groupby(['PROV_', 'DEPTO_', 'FRACC_']).apply(lambda x: x.nlargest(1, 'area')).reset_index(drop = True)[['PROV_', 'DEPTO_', 'FRACC_', 'in1']]
# matches['PROV_DEPTO_'] = matches['PROV_'] + matches['DEPTO_']

## *
spat_join = fracs_in_depto.groupby(['PROV_', 'DEPTO_', 'in1']).size().reset_index()
spat_join = spat_join.rename(columns = {0: 'n_fracs_in_depto'})

### Si en result los codigos son iguales, asociarlas.
## Esto es porque los DPTOS que se subdividen cambian el codigo. 
## Ademas, asi se evitan asociaciones espurias debidas a desalineamientos. La excepcion son las comunas de CABA, pero con esos no hay problemas de desalineaminto

dpto_codes_match_confirmados_2 = spat_join.loc[(spat_join.PROV_ + spat_join.DEPTO_) == spat_join.in1].drop('n_fracs_in_depto', 1) 
## Con que una fraccion este en un in1 del mismo codigo, ya todas las fracciones vienen asignadas a ese codigo en comun.

dpto_codes_match_confirmados_3 = pd.concat([dpto_codes_match_confirmados_1, dpto_codes_match_confirmados_2])

## Usar las intersecciones en los casos donde no coinciden los codigos
fracs_in_depto_ = fracs_in_depto.loc[~fracs_in_depto['in1'].isin(dpto_codes_match_confirmados_3['in1'])]
# ### Las Fracciones que quedan son de Chascomus, TdF y CABA

In [ ]:
### El resultado final es
result = censo[['PROV_','DEPTO_','FRACC_']].merge(fracs_in_depto_, how = 'left')
result['in1'] = result['in1'].fillna(result.PROV_ + result.DEPTO_)

In [ ]:
result.head()

In [ ]:
test = result.groupby(['PROV_','DEPTO_']).in1.nunique()
test = test.loc[test > 1].reset_index()
test = test.drop('in1', 1).merge(spat_join, on = ['PROV_','DEPTO_'])
test.merge(ref_[['in1', 'nam']])#.sort_values(['PROV_', 'DEPTO_', 'FRACC_'])

In [ ]:
cross = test.merge(ref_[['in1', 'nam', 'geometry']], on = 'in1').sort_values(['PROV_', 'DEPTO_'])
cross.to_csv('./cross_'+year+'_IGN.csv', index = False)
cross

In [ ]:
xx

In [ ]:
provs_gdf = ref_[['PROV_', 'geometry']].dissolve('PROV_')

In [ ]:
ref_provs = gpd.read_file('./../chequeo_geo/provincia_IGN/')
ref_provs = ref_provs.to_crs(CONICET_radios.crs)
ref_provs['PROV_'] = ref_provs['in1'].str[:2]
ref_provs['DEPTO_'] = ref_provs['in1'].str[2:]
# ref_provs.head()

In [ ]:
for i, row in test[['PROV_', 'DEPTO_']].drop_duplicates().iterrows():
    
    fig, ax = plt.subplots(1, figsize = (10, 10))
    
    # select
    censo_ = result.loc[(result.PROV_ == row.PROV_) & (result.DEPTO_ == row.DEPTO_)]
    dpto_gdf = gpd.GeoDataFrame(censo_[['PROV_', 'DEPTO_']].drop_duplicates().merge(CONICET_dptos))
    dpto_gdf.plot(ax = ax, color = 'None', edgecolor = 'k', lw = 1.5)
    
    gpd.GeoDataFrame(censo_.merge(CONICET_fracs)).plot('in1', ax = ax, alpha = .4, edgecolor = 'k')
    
    ign_gdf = pd.DataFrame(row).T.merge(test).merge(ref_[['in1', 'nam', 'geometry']], on = 'in1')
    ign_gdf = gpd.GeoDataFrame(ign_gdf)
    ign_gdf.plot(ax = ax, color = 'None', edgecolor = 'g', lw = 1.5)
    
    # plot bounds
    x0, y0, x1, y1 = dpto_gdf.geometry.unary_union.bounds
    ax.set_xlim(x0, x1); ax.set_ylim(y0, y1)
    ax.axis('off')
    
    display(ign_gdf[['PROV_','DEPTO_','in1', 'nam']])
    plt.show()

In [ ]:
# ## Como no son muchas, se pueden hacer sin loop provincial.

# inters = gpd.overlay(fracs_ubicar, ref_[['in1', 'geometry']], how='intersection') 
# inters['area'] = inters.area

# ## La fraccion se asigna al depto donde este mayor parte de su area.
# fracs_in_depto = inters.groupby(['PROV_', 'DEPTO_', 'FRACC_']).apply(lambda x: x.nlargest(1, 'area')).reset_index(drop = True)[['PROV_', 'DEPTO_', 'FRACC_', 'in1']]
# # matches['PROV_DEPTO_'] = matches['PROV_'] + matches['DEPTO_']

# spat_join = censo[['PROV_','DEPTO_','FRACC_']].merge(fracs_in_depto, how = 'left')
# spat_join['in1'] = spat_join['in1'].fillna(spat_join.PROV_ + spat_join.DEPTO_)
